# Food Delivery Analysis - zaara shaikh
## Innomatics Research Lab enterance test

In [1]:
#importing essential libraries
import pandas as pd
import sqlite3
import json

**Step 1: Loading CSV data**

In [2]:
df_orders = pd.read_csv('raw_data\orders.csv')
print("orders.csv loaded successfully!")

orders.csv loaded successfully!


**Step 2: Loading JSON data**

In [3]:
df_users = pd.read_json(r'raw_data\users.json')
print("users.json loaded successfully!")

users.json loaded successfully!


**Step 3: Loading SQL data**

In [4]:
#connecting to sqlite3
conn = sqlite3.connect(':memory:') #in-memory database
with open(r'raw_data\restaurants.sql', 'r') as f:
    sql_script = f.read()
    conn.executescript(sql_script) #runnning the sql script

In [5]:
df_restaurants = pd.read_sql('SELECT *FROM RESTAURANTS', conn)
print(" restaurants.sql loaded successfully!")

 restaurants.sql loaded successfully!


**Step 4: Merging data**

In [6]:
# merging orders + users on 'userid' (left join)
df_merged = pd.merge(df_orders, df_users, on = 'user_id', how = 'left') 

In [7]:
# merging result + restaurants on 'userid' (left join)
df_final = pd.merge(df_merged, df_restaurants, on = 'restaurant_id', how = 'left')

In [8]:
print(f"Final Dataset created! \n shape:{df_final.shape}")

Final Dataset created! 
 shape:(10000, 12)


In [9]:
df_final.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


**Data Integrity Checks**

In [10]:
# displaying dataframe infromations(column datatypes etc)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [11]:
# changing date datatype from object to datetime
df_final['order_date'] = pd.to_datetime(df_final['order_date'], dayfirst = True)
print(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   order_id           10000 non-null  int64         
 1   user_id            10000 non-null  int64         
 2   restaurant_id      10000 non-null  int64         
 3   order_date         10000 non-null  datetime64[ns]
 4   total_amount       10000 non-null  float64       
 5   restaurant_name_x  10000 non-null  object        
 6   name               10000 non-null  object        
 7   city               10000 non-null  object        
 8   membership         10000 non-null  object        
 9   restaurant_name_y  10000 non-null  object        
 10  cuisine            10000 non-null  object        
 11  rating             10000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 937.6+ KB
None


In [12]:
# amount can't be negative
invalid_amounts = df_final[df_final['total_amount']<=0]  
if not invalid_amounts.empty:
    print(f"FOUND {llen(invalid_amounts)} orders with invalid amounts (<=0)")
else:
    print(f"Integrity passed, All order amounts are positive")

Integrity passed, All order amounts are positive


In [13]:
# ratings must be in between 0-5
invalid_ratings = df_final[(df_final['rating']<0)|(df_final['rating']>5)]  
if not invalid_ratings.empty:
    print(f"FOUND {llen(invalid_rating)} restaurants with invalid ratings (<0 or >5)")
else:
    print(f"Integrity passed, All ratings are between 0-5")

Integrity passed, All ratings are between 0-5


In [14]:
# to check memberships ie either Gold or Regular only
df_final['membership'].nunique()

2

In [15]:
# check duplicate rows, and prevent redundancy
if df_final.duplicated().sum() == 0:  
    print("Integrity Passed, No duplicate Rows found")
else:
    print(f"FOUND {df_final.duplicated().sum()} duplicate rows")

Integrity Passed, No duplicate Rows found


**Step 5: Exporting Final dataset to csv**

In [16]:
df_final.to_csv('final_food_delivery_dataset.csv', index = False)
print("'final_food_delivery_dataset.csv' has been saved to the working directory!")

'final_food_delivery_dataset.csv' has been saved to the working directory!


**SECTION 1: MCQs**

## Q1: City which has the highest total revenue (total_amount) from Gold members

In [17]:
df_final.to_sql('food_delivery', conn, if_exists = 'replace', index= False)

query1 = """
SELECT city, SUM(total_amount) AS total_revenue FROM food_delivery
WHERE membership = "Gold"
GROUP BY city
ORDER BY total_revenue DESC
LIMIT 1;
"""
result = pd.read_sql(query1, conn)
print(result)

      city  total_revenue
0  Chennai     1080909.79


## Q2: Which cuisine has the highest average order value across all orders?

In [18]:
query2 = """
SELECT cuisine, ROUND(AVG(total_amount),2) AS avg_order_value, COUNT(*) AS num_orders
FROM food_delivery
GROUP BY cuisine
ORDER BY avg_order_value DESC
LIMIT 1;
"""
result = pd.read_sql(query2, conn)
print(result)

   cuisine  avg_order_value  num_orders
0  Mexican           808.02        2581


## Q3: How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

In [19]:
query3 = """
SELECT COUNT(DISTINCT user_id) AS distinct_users
FROM 
    ( SELECT user_id, SUM(total_amount) AS user_total
    FROM food_delivery
    GROUP BY user_id
    HAVING SUM(total_amount)>1000
    );
"""
result = pd.read_sql(query3, conn)
print(result)

   distinct_users
0            2544


## Q4: Which restaurant rating range generated the highest total revenue?

In [20]:
query4 = """
SELECT 
    CASE
        WHEN rating>=3.0 AND rating <=3.5 THEN '3.0-3.5'
        WHEN rating>3.5 AND rating <=4.0 THEN '3.5-4.0'
        WHEN rating>4.0 AND rating <=4.5 THEN '4.1-4.5'
        WHEN rating>4.5 AND rating <=5.0 THEN '4.6-5.0'
    END AS rating_range,
    SUM(total_amount) AS total_revenue
FROM food_delivery
GROUP BY rating_range
ORDER BY total_revenue DESC;
"""
result = pd.read_sql(query4, conn)
print(result)

  rating_range  total_revenue
0      4.6-5.0     2197030.75
1      3.0-3.5     2136772.70
2      4.1-4.5     1960326.26
3      3.5-4.0     1717494.41


## Q5: Among Gold members, which city has the highest average order value?

In [21]:
query5 = """
SELECT city, ROUND(AVG(total_amount),2) AS avg_order_value, COUNT(*) AS num_orders
FROM food_delivery
WHERE membership = 'Gold'
GROUP BY city
ORDER BY avg_order_value DESC;
"""
result = pd.read_sql(query5, conn)
print(result)

        city  avg_order_value  num_orders
0    Chennai           808.46        1337
1  Hyderabad           806.42        1112
2  Bangalore           793.22        1254
3       Pune           781.16        1284


## Q6: Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [22]:
query6 = """
SELECT cuisine, COUNT(DISTINCT restaurant_id) AS num_restaurants, SUM(total_amount) AS total_revenue, COUNT(*) AS num_orders
FROM food_delivery
GROUP BY cuisine
ORDER BY num_restaurants ASC, total_revenue DESC;
"""
result = pd.read_sql(query6, conn)
print(result)

   cuisine  num_restaurants  total_revenue  num_orders
0  Chinese              120     1930504.65        2418
1  Italian              126     2024203.80        2532
2   Indian              126     1971412.58        2469
3  Mexican              128     2085503.09        2581


## Q7: What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

In [23]:
query7 = """
SELECT 
    ROUND((COUNT(CASE WHEN membership = 'Gold' THEN 1 END)*100.0/COUNT(*)),
    0)
    AS gold_percentage
FROM food_delivery;
"""
result = pd.read_sql(query7, conn)
print(result)

   gold_percentage
0             50.0


## Q8: Which restaurant has the highest average order value but less than 20 total orders?

In [24]:
query8 = """
SELECT 
        restaurant_name_x AS restaurant_name,
        COUNT(*) AS total_orders,
        ROUND(AVG(total_amount),2) AS avg_order_value
FROM food_delivery
WHERE restaurant_name_x IN ('Grand Cafe Punjabi','Grand Restaurant South Indian','Ruchi Mess Multicuisine','Ruchi Foods Chinese')
GROUP BY restaurant_name_x
--HAVING COUNT(*) < 20 
ORDER BY avg_order_value DESC
LIMIT 10;
"""
result = pd.read_sql(query8, conn)
print(result)

                 restaurant_name  total_orders  avg_order_value
0        Ruchi Mess Multicuisine            40           851.23
1  Grand Restaurant South Indian            29           842.57
2             Grand Cafe Punjabi            32           765.41
3            Ruchi Foods Chinese            19           686.60


## Q9: Which combination contributes the highest revenue?

In [26]:
query9 = """
SELECT 
    membership || '+' || cuisine AS combination,
    SUM(total_amount) AS total_revenue,
    COUNT(*) AS num_orders
FROM food_delivery
GROUP BY membership, cuisine
ORDER BY total_revenue DESC
"""
result = pd.read_sql(query9, conn)
print(result)

       combination  total_revenue  num_orders
0  Regular+Mexican     1072943.30        1320
1  Regular+Italian     1018424.75        1256
2     Gold+Mexican     1012559.79        1261
3     Gold+Italian     1005779.05        1276
4   Regular+Indian      992100.27        1233
5      Gold+Indian      979312.31        1236
6     Gold+Chinese      977713.74        1214
7  Regular+Chinese      952790.91        1204


## Q10: During which quarter of the year is the total revenue highest?

In [27]:
query10 = """
SELECT 
    CASE 
        WHEN CAST(SUBSTR(order_date,6,2) AS INTEGER) BETWEEN 1 AND 3 THEN 'Q1 (Jan-Mar)'
        WHEN CAST(SUBSTR(order_date,6,2) AS INTEGER) BETWEEN 4 AND 6 THEN 'Q2 (Apr-Jun)'
        WHEN CAST(SUBSTR(order_date,6,2) AS INTEGER) BETWEEN 7 AND 9 THEN 'Q3 (Jul-Sep)'
        WHEN CAST(SUBSTR(order_date,6,2) AS INTEGER) BETWEEN 10 AND 12 THEN 'Q4 (Oct-Dec)'
    END AS quarter,
    SUM(total_amount) AS total_revenue,
    COUNT(*) AS num_orders
FROM food_delivery
GROUP BY quarter
ORDER BY total_revenue DESC;
"""
result = pd.read_sql(query10, conn)
print(result)

        quarter  total_revenue  num_orders
0  Q3 (Jul-Sep)     2037385.10        2522
1  Q4 (Oct-Dec)     2018263.66        2519
2  Q1 (Jan-Mar)     2010626.64        2519
3  Q2 (Apr-Jun)     1945348.72        2440


**SECTION 2: NUMERICAL ANSWER QUESTIONS**

## Q1 How many total orders were placed by users with Gold membership?

In [28]:
query_num1 = """
SELECT COUNT(*) AS total_orders
FROM food_delivery
WHERE membership = 'Gold' ;
"""
result = pd.read_sql(query_num1, conn)
print(result)

   total_orders
0          4987


## Q2: What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

In [29]:
query_num2 = """
SELECT ROUND(SUM(total_amount),0) AS total_revenue
FROM food_delivery
WHERE city = 'Hyderabad';
"""
result = pd.read_sql(query_num2, conn)
print(result)

   total_revenue
0      1889367.0


## Q3: How many distinct users placed at least one order?

In [30]:
query_num3 = """
SELECT COUNT(DISTINCT user_id) AS distinct_users
FROM food_delivery;
"""
result = pd.read_sql(query_num3, conn)
print(result)

   distinct_users
0            2883


## Q4: What is the average order value (rounded to 2 decimals) for Gold members?

In [31]:
query_num4 = """
SELECT ROUND(AVG(total_amount),2) AS avg_order_value
FROM food_delivery
WHERE membership = 'Gold';
"""
result = pd.read_sql(query_num4, conn)
print(result)

   avg_order_value
0           797.15


## Q5: How many orders were placed for restaurants with rating ≥ 4.5?

In [32]:
query_num5 = """
SELECT COUNT(*) AS total_orders
FROM food_delivery
WHERE rating>=4.5;
"""
result = pd.read_sql(query_num5, conn)
print(result)

   total_orders
0          3374


## Q6: How many orders were placed in the top revenue city among Gold members only?

In [33]:
query_num6 = """
SELECT COUNT(*) AS total_orders
FROM food_delivery
WHERE membership = 'Gold' AND city = (
    SELECT city FROM food_delivery
    WHERE membership = 'Gold'
    GROUP BY city
    ORDER BY SUM(total_amount) DESC
    LIMIT 1
    );
"""
result = pd.read_sql(query_num6, conn)
print(result)

   total_orders
0          1337
